# Label DP SGD (Post Scale)

This notebook walks through how to train a model to recognize hand written
digits using label differentially private gradient decent and the MNIST dataset.
In this setting, one party has the images and the other party has the labels.
They would like to collaborate to train a model without revealing their data.

This colab uses the post-scale approach to training.

Before starting, install the tf-shell package.

```bash
pip install tf-shell
```

First, import some modules and set up tf-shell. The parameters are for the SHELL
encryption library, which tf-shell uses, and mostly depend on the multiplicative
depth of the computation to be performed. This example performs back
propagation, thus the multiplicative depth is determined by the number of
layers. For more information, see [SHELL](https://github.com/google/shell).

In [1]:
import time
from datetime import datetime
import tensorflow as tf
import keras
import numpy as np
import tf_shell
import tf_shell_ml
import os

2024-06-10 21:55:22.590363: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 21:55:22.590673: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:55:22.592479: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:55:22.618681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 21:55:23.196862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
# Num plaintext bits: 19, noise bits: 40
# Max representable value: 61895
context = tf_shell.create_context64(
    log_n=11,
    main_moduli=[576460752303439873],
    plaintext_modulus=557057,
    scaling_factor=3,
    mul_depth_supported=1,
)
# 121 bits of security according to lattice estimator primal_bdd.

# Create the secret key for encryption and a rotation key (rotation key is
# an auxilary key required for operations like roll or matmul).
secret_key = tf_shell.create_key64(context)
public_rotation_key = tf_shell.create_rotation_key64(context, secret_key)

# The batch size is determined by the ciphertext parameters, specifically the
# schemes polynomial's ring degree because tf-shell uses batch axis packing.
# Furthermore, two micro-batches to run in parallel.
batch_size = context.num_slots

Setup MNIST dataset.

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = np.reshape(x_train, (-1, 784)), np.reshape(x_test, (-1, 784))
x_train, x_test = x_train / np.float32(255.0), x_test / np.float32(255.0)
y_train, y_test = tf.one_hot(y_train, 10), tf.one_hot(y_test, 10)

epochs = 1
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_dataset.shuffle(buffer_size=2048)
    .batch(batch_size, drop_remainder=True)
    .repeat(count=epochs)
)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(batch_size, drop_remainder=True)

Create a simple model with a hidden layer of size 64 and an output layer
of size 10 (for each of the 10 digits).

In [4]:
mnist_layers = [
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="sigmoid"),
]

model = keras.Sequential(mnist_layers)
model.compile(
    optimizer="adam",
    metrics=["accuracy"],
)

Next, define the `train_step` function which will be called for each batch on an
encrypted batch of labels, y. The function first does a forward on the plaintext
image x to compute a predicted label, then does backpropagation using the
encrypted label y.

In [5]:
@tf.function
def train_step(x, y):
    """One step of training with using the "post scale" approach.

    High level idea:
    For each output class, backprop to compute the gradient but exclude the loss
    function. Now we have a _vector_ of model updates for one sample. The real
    gradient update for the sample is a linear combination of the vector of
    weight updates whose scale is determined by dJ_dyhat (the derivative of the
    loss with respect to the predicted output yhat). Effectively, we have
    factored out dJ_dyhat from the gradient. Separating out dJ_dyhat allows us
    to scale the weight updates easily when the label is secret and the gradient
    must be computed under encryption / multiparty computation because the
    multiplicative depth of the computation is 1, however the number of
    multiplications required now depends on the model size AND the number of
    output classes. In contrast, standard backpropagation only requires
    multiplications proportional to the model size, howver the multiplicative
    depth is proportional to the model depth.
    """

    # Unset the activation function for the last layer so it is not used in
    # computing the gradient. The effect of the last layer activation function
    # is factored out of the gradient computation and accounted for below.
    model.layers[-1].activation = tf.keras.activations.linear

    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)  # forward pass
    grads = tape.jacobian(y_pred, model.trainable_weights)
    # ^  layers list x (batch size x num output classes x weights) matrix
    # dy_pred_j/dW_sample_class


    # Reset the activation function for the last layer and compute the real
    # prediction.
    model.layers[-1].activation = tf.keras.activations.sigmoid
    y_pred = model(x, training=False)

    # Compute y_pred - y (where y is encrypted).
    scalars = y.__rsub__(y_pred)  # dJ/dy_pred
    # ^  batch_size x num output classes.

    # Expand the last dim so that the subsequent multiplication is
    # broadcasted.
    scalars = tf_shell.expand_dims(scalars, axis=-1)
    # ^ batch_size x num output classes x 1

    # Scale each gradient. Since 'scalars' may be a vector of ciphertexts, this
    # requires multiplying plaintext gradient for the specific layer (2d) by the
    # ciphertext (scalar). To do so efficiently under encryption requires
    # flattening and packing the weights, as shown below.
    ps_grads = []
    for layer_grad_full in grads:
        # Remember the original shape of the gradient in order to unpack them
        # after the multiplication so they can be applied to the model.
        batch_sz = layer_grad_full.shape[0]
        num_output_classes = layer_grad_full.shape[1]
        grad_shape = layer_grad_full.shape[2:]

        packable_grad = tf.reshape(layer_grad_full, [batch_sz, num_output_classes, -1])
        # ^  batch_size x num output classes x flattened weights

        # Scale the gradient precursors.
        scaled_grad = scalars * packable_grad
        # ^ dJ/dW = dJ/dy_pred * dy_pred/dW 

        # Sum over the output classes.
        scaled_grad = tf_shell.reduce_sum(scaled_grad, axis=1)
        # ^  batch_size x 1 x flattened weights

        # In the real world, this approach would also likely require clipping
        # the gradient, aggregation, and adding DP noise.

        # Reshape to remove the '1' dimension in the middle.
        scaled_grad = tf_shell.reshape(scaled_grad, [batch_sz] + grad_shape)
        # ^  batch_size x weights

        # Sum over the batch.
        scaled_grad = tf_shell.reduce_sum(scaled_grad, axis=0, rotation_key=public_rotation_key)
        # ^  batch_size x flattened weights
        # Every [i, ...] is the same, the sum over the batching dim axis=0.

        ps_grads.append(scaled_grad)

    return ps_grads


@tf.function
def train_step_wrapper(x_batch, y_batch):
    # Encrypt
    enc_y_batch = tf_shell.to_encrypted(y_batch, secret_key, context)

    # Train
    ps_grads = train_step(x_batch, enc_y_batch)

    # Decrypt
    batch_sz = context.num_slots
    top_grads = [tf_shell.to_tensorflow(enc_g, secret_key)[0] for enc_g in ps_grads]
    bottom_grads = [tf_shell.to_tensorflow(enc_g, secret_key)[batch_sz // 2] for enc_g in ps_grads]
    # ^ take the first element of each batch because the grad sum is repeated over the batching dim.

    model.optimizer.apply_gradients(
        zip(
            top_grads,
            model.trainable_weights
        )
    )
    model.optimizer.apply_gradients(
        zip(
            bottom_grads,
            model.trainable_weights
        )
    )

Here is the training loop. Each inner iteration runs a batch of size 2^(11),
then meaures the model accuracy.

Tensorboard can be used to visualize the training progress. See cell output for
command to start tensorboard.

In [6]:
start_time = time.time()

# Set up tensorboard logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.abspath("") + "/tflogs/post-scale-%s" % stamp
print(f"To start tensorboard, run: tensorboard --logdir /tmp/tflogs --host 0.0.0.0")
print(f"\ttensorboard profiling requires: pip install tensorboard_plugin_profile")
writer = tf.summary.create_file_writer(logdir)

# Iterate over the batches of the dataset.
for step, (x_batch, y_batch) in enumerate(train_dataset.take(batch_size)):
    print(
        f"Batch: {step} / {len(train_dataset)}, Time Stamp: {time.time() - start_time}"
    )

    if step == 0:
        tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=logdir)

    train_step_wrapper(x_batch, y_batch)

    if step == 0:
        with writer.as_default():
            tf.summary.trace_export(name="label_dp_sgd_post_scale", step=step)

    # Check the accuracy.
    average_loss = 0
    average_accuracy = 0
    for x, y in val_dataset:
        y_pred = model(x, training=False)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, y_pred))
        accuracy = tf.reduce_mean(
            tf.cast(
                tf.equal(tf.argmax(y, axis=1), tf.argmax(y_pred, axis=1)), tf.float32
            )
        )
        average_accuracy += accuracy
        average_loss += loss
    average_loss /= len(val_dataset)
    average_accuracy /= len(val_dataset)
    tf.print(f"\taccuracy: {accuracy}")

    with writer.as_default():
        tf.summary.scalar("loss", average_loss, step=step)
        tf.summary.scalar("accuracy", average_accuracy, step=step)


print(f"Total training time: {time.time() - start_time} seconds")

To start tensorboard, run: tensorboard --logdir /tmp/tflogs --host 0.0.0.0
	tensorboard profiling requires: pip install tensorboard_plugin_profile
Batch: 0 / 29, Time Stamp: 0.07114076614379883


2024-06-10 21:55:24.765112: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-06-10 21:55:24.765142: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


	accuracy: 0.07861328125


2024-06-10 21:56:43.385829: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-06-10 21:56:43.396207: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-06-10 21:56:43.397087: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: /workspaces/tf-shell/examples/tflogs/post-scale-20240610-215524/plugins/profile/2024_06_10_21_56_43/e64b0b6b3843.xplane.pb
2024-06-10 21:56:43.457550: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Batch: 1 / 29, Time Stamp: 78.8082582950592
	accuracy: 0.0966796875


...

2024-06-10 22:21:44.155515: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Batch: 27 / 29, Time Stamp: 1579.4650423526764
	accuracy: 0.84228515625


2024-06-10 22:22:40.935333: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Batch: 28 / 29, Time Stamp: 1636.244945526123
	accuracy: 0.84423828125
Total training time: 1693.0756685733795 seconds


2024-06-10 22:23:37.765942: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 22:23:37.768871: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
